In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import warnings
import time
warnings.filterwarnings(action='ignore')

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the website containing cocktails
url = 'https://www.thecocktailproject.com/search-recipes/'

# Open the website
driver.get(url)

# Wait for the page to load and for the cookie popup to disappear
wait = WebDriverWait(driver, 10)
wait.until(EC.invisibility_of_element_located((By.ID, 'onetrust-pc-btn-handler')))

# Initialize an empty dictionary to store cocktail names, links, and image links
cocktails_dict = {}

# Loop through each page from 1 to 22
for i in range(0, 22):
    # Get the URL of the page
    page_url = f"https://www.thecocktailproject.com/search-recipes/?page={i}"
    # Visit the page
    driver.get(page_url)
    
    # Wait for the cocktail elements to be present
    cocktail_elements = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '.receipe-grid-items')))

    # Loop through each cocktail element on the current page
    for cocktail_element in cocktail_elements:
        # Extract name of the cocktail
        name = cocktail_element.find_element(By.CSS_SELECTOR, '.item-desc a').text.strip()

        # Extract link of the cocktail
        link = cocktail_element.find_element(By.CSS_SELECTOR, '.item-detail a').get_attribute('href')

        # Extract image link of the cocktail
        image_element = cocktail_element.find_element(By.CSS_SELECTOR, '.field--name-field-recipe-image img')
        image_link = image_element.get_attribute('src')
        
        
        # Add the name, link, and image link of the cocktail to the dictionary
        cocktails_dict[name] = {'link': link, 'image_link': image_link}

# Close the browser
driver.quit()


In [4]:
# 만들어뒀던 dict 로 이미지 다운로드, 이미지 이름 문제 있는 것들 renaming 해서 그것도 다시 새로운 dict image_names에 저장
import os
import requests

directory = "images_halways"
image_names = {}
if not os.path.exists(directory):
    os.makedirs(directory)
    
# Loop through each cocktail in the dictionary
for name, image_link in cocktails_dict.items():
    
    # Save image as PNG with cocktail name as file name
    response = requests.get(image_link)
    if response.status_code == 200:
        # Remove special characters from the name
       
        image_name = name
        image_name = image_name.replace("®", "")
        image_name = image_name.replace("\n","")
        image_names[name] = image_name
        
        # Define the file path
        file_path = f"images/{image_name}.png"
        
        # Write the new image to file
        with open(file_path, "wb") as f:
            f.write(response.content)


InvalidSchema: No connection adapters were found for "{'link': 'https://www.thecocktailproject.com/Espresso-Martini', 'image_link': 'https://www.thecocktailproject.com/sites/default/files/styles/grid-thumb/public/EspressoMartini.jpg?itok=161L5upy'}"

In [5]:
# 이미지 총 개수 440인거 확인
directory = "images"
files = os.listdir(directory)
num_files = len(files)

# Print the number of files
print(f"Number of files in {directory}: {num_files}")

Number of files in images: 440


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Create an empty dictionary to store ingredients for each cocktail
cocktails_with_ingredients = {}

for name, data in cocktails_dict.items():
    link = data['link']
    driver.get(link)

    # Wait for the page to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'solid-block')))

    # Find all div elements with the class "title" within the solid block
    title_elements = driver.find_elements(By.CSS_SELECTOR, '.solid-block .title')

    # Extract ingredient names from the title elements and replace special characters
    #ingredient_names = [title.text.strip().replace('½', '1/2').replace('®', '').replace('¼', '1/4').replace('\n', ' ') for title in title_elements]
    # Extract ingredient names from the title elements and replace special characters
    ingredient_names = [title.text.strip().replace('½', '1/2').replace('®', '').replace('¼', '1/4').replace('\n', ' ') for title in title_elements if title.text.strip()]

    # Assign the ingredient list to the current cocktail
    data['ingredients'] = ingredient_names
    
    # Add the updated cocktail data to the new dictionary
    cocktails_with_ingredients[name] = data

# Update cocktails_dict with the new dictionary containing ingredient information
cocktails_dict.update(cocktails_with_ingredients)

# Close the browser
driver.quit()



KeyboardInterrupt



In [145]:
import csv

# Define the file name for the CSV file
csv_file = 'cocktails_halways.csv'

# Define the field names for the CSV file
field_names = ['Cocktail Name', 'Link', 'Image Link', 'Ingredients']

# Write the cocktails_dict to the CSV file
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=field_names)
    
    # Write the header row
    writer.writeheader()
    
    # Write each cocktail entry
    for cocktail_name, cocktail_info in cocktails_dict.items():
        # Replace ® symbol with empty string in cocktail name
        cocktail_name = cocktail_name.replace('®', '')
        
        # Check if the 'ingredients' key exists
        if 'ingredients' in cocktail_info:
            # If the key exists, write the ingredients as a comma-separated string
            writer.writerow({
                'Cocktail Name': cocktail_name,
                'Link': cocktail_info['link'],
                'Image Link': cocktail_info['image_link'],
                'Ingredients': ', '.join(cocktail_info['ingredients'])
            })
        else:
            # If the key does not exist, write an empty string for ingredients
            writer.writerow({
                'Cocktail Name': cocktail_name,
                'Link': cocktail_info['link'],
                'Image Link': cocktail_info['image_link'],
                'Ingredients': ''
            })

print("CSV file created successfully!")


TypeError: list indices must be integers or slices, not str

In [154]:
import csv

# Function to filter out unwanted ingredients
def filter_ingredients(ingredient_list):
    # Words to check for in ingredient names
    unwanted_words = ['How', 'When', 'What', 'Do', 'Where', 'Why', 'when', 'how', 'why', 'what', 'Cocktails', 'Cocktail', 'Home', 'home', 'Guide', 'Your', 'Bar', 'Party', 'Is', 'Should', 'vs.', 'The']
    # Filter out ingredients that contain unwanted words
    filtered_ingredients = [ingredient.strip() for ingredient in ingredient_list if not any(word in ingredient for word in unwanted_words)]
    return filtered_ingredients

# Define the file names for the input and output CSV files
input_csv_file = 'cocktails_halways.csv'
output_csv_file = 'filtered_cocktails_halways.csv'

# Read the CSV file and filter ingredients
filtered_cocktails_dict = {}
with open(input_csv_file, 'r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        cocktail_name = row['Cocktail Name']
        ingredients = row['Ingredients'].split(', ')
        filtered_ingredients = filter_ingredients(ingredients)
        filtered_cocktails_dict[cocktail_name] = filtered_ingredients

# Write the filtered data to a new CSV file
with open(output_csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Cocktail Name', 'Ingredients'])
    writer.writeheader()
    for cocktail_name, ingredients in filtered_cocktails_dict.items():
        writer.writerow({'Cocktail Name': cocktail_name, 'Ingredients': ', '.join(ingredients)})

print("Filtered cocktails saved to:", output_csv_file)


Filtered cocktails saved to: filtered_cocktails7.csv
